# Redes neuronales (ejercicio)

### **Importante: comentar adecuadamente cada paso realizado**, relacionándolo con lo visto en la teoría.

## Parte 1: aplicación de redes neuronales a clasificación (análisis de sentimientos)

Se pide aplicar un modelo de redes neuronales al problema de decidir si una crítica de cine es positiva o negativa. Para ello volvemos a usar los datos de IMDB (Internet Movie Database) que vimos en el módulo 2 (modelo probabilístico).

Hacerlo usando los dos sistemas vistos, comparando los resultados:
* Scikit learn: usar `MLPClassifier`
* Keras con Tensorflow: usar `Sequential` y capas tipo `Dense` con la arquitectura adecuda.


Aunque ya hemos visto que los datos están disponibles en http://ai.stanford.edu/~amaas/data/sentiment/ , en este caso pedimos cargar los datos usando la utilidad `imdb`de Keras. Se puede consultar en el manual de Keras: https://keras.io/datasets/ Cargarlos con `imdb.load_data` y usar los datos cargados como punto de partida a este ejercicio (tanto para su aplicar scikit learn como para aplicar keras). Prestar atención al formato en el que se cargar, que no es el mismo que hamos visto hasta ahora.  

Los textos han de ser vectorizados para que se puedan ser procesados por una red. Para esto, tenemos varias alternativas, usar una de ellas:

* Vectorizando "manualmente", definiendo una función en python que lo haga.
* Vectorizadores de scikit learn (ya vistos)
* Herramientas de vectorización de keras: https://keras.io/preprocessing/text/

Mostrar algunas pruebas realizadas con distintas arquitecturas y/o hiperparámetros. No es necesario ser muy exhaustivo ni usar `GridSearchCV` en scikit learn ni el equivalente en keras. Tan solo mostrar alguna experimentación y ajuste manual.  

### Ejercicio
Empecemos por cargar los datos de las críticas de `imdb` utilizando Keras. El método `load_data` devuelve dos tuplas de arrays: uno con el conjunto de entrenamiento y otro con el conjunto de prueba. Cada tupla tiene los datos (*X_train_code*, *X_test_code*) que son una lista de índices, y su correspondiente clasificación (*y_train*, *y_test*) en 1 (positivo) y 0 (negativo). 

In [1]:
# Importar datos
import tensorflow as tf 
from tensorflow import keras
(X_train_code, y_train), (X_test_code, y_test) = keras.datasets.imdb.load_data()
X_train_code.shape, X_test_code.shape

((25000,), (25000,))

La lista de índices de los datos son palabras codificadas, es decir, cada índice es una palabra codificada, por tanto una crítica completa es el conjunto de todas los índices de las palabras. Como queremos trabajar con las críticas, vamos a decodificarlas, tanto el conjunto de entrenamiento como el de prueba. Para ello utilizamos el método `get_word_index` con el que obtenemos el diccionario de palabras y el índice correspondiente a cada una de ellas. 

In [2]:
# Decodificar
word_index = keras.datasets.imdb.get_word_index()
inverted_word_index = dict((i, word) for (word, i) in word_index.items())

X_train_decode = []
for critic in X_train_code:
    X_train_decode.append(" ".join([inverted_word_index.get(i, '') for i in critic]))

X_test_decode = []
for critic in X_test_code:
    X_test_decode.append(" ".join([inverted_word_index.get(i, '') for i in critic]))

Para procesar los textos que hemos decodificado es necesario vectorizarlos. Vamos a aplicar los vectorizadores de scikit learn que hemos visto en sesiones anteriores. Primero entrenamos el vectorizador y a continuación aplicamos la vectorización a ambos grupos, para obtener los conjuntos con los que vamos a trabajar.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(X_train_decode)
X_train = vect.transform(X_train_decode)
X_test = vect.transform(X_test_decode)

Ahora que tenemos los datos, vamos a aplicar dos modelo de redes neuronales. Empecemos primero por el de scikit_learn `MLPClassifier` dentro del módulo `neural_network`, que permitirá usar las redes multicapa hacia adelante. Vamos a ver el rendimiento que obtenemos con los valores por defecto, una capa oculta y 100 unidades en ese capa, y con el método `lbfgs`, ya que no necesita tantos ajustes y con los datos que manejamos (25.000 ejemplos) se espera que funcione relativamente bien.

In [4]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver="lbfgs", max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

print("Rendimiento en entrenamiento: {:.2f}".format(mlp.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.2f}".format(mlp.score(X_test, y_test)))

Rendimiento en entrenamiento: 1.00
Rendimiento en el conjunto de prueba: 0.86


El rendimiento obtenido no es muy bueno, de hecho se puede ver que sobre el conjunto de entrenamiento se produce algo de sobreajuste. Vamos a aplicar regularización, mediente el parámetro *alpha*, a ver si conseguimos que mejore algo el rendimiento.

In [5]:
mlp_a1 = MLPClassifier(solver="lbfgs", max_iter=1000, alpha=1, random_state=42)
mlp_a1.fit(X_train, y_train)
print("Rendimiento en entrenamiento: {:.2f}".format(mlp_a1.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.2f}".format(mlp_a1.score(X_test, y_test)))

mlp_a10 = MLPClassifier(solver="lbfgs", max_iter=1000, alpha=0.05, random_state=42)
mlp_a10.fit(X_train, y_train)
print("Rendimiento en entrenamiento: {:.2f}".format(mlp_a10.score(X_train, y_train)))
print("Rendimiento en el conjunto de prueba: {:.2f}".format(mlp_a10.score(X_test, y_test)))

Rendimiento en entrenamiento: 1.00
Rendimiento en el conjunto de prueba: 0.87
Rendimiento en entrenamiento: 1.00
Rendimiento en el conjunto de prueba: 0.86


A pesar de aplicar regularización, se obtienen resultados similares.<br><br>
Veamos ahora qué resultados obtenemos aplicando *Keras* con Tensorflow usando el módulo `Sequential` y capas tipo `Dense`. Cada unidad de estas capas recibe tantas conexiones como unidades en la capa anterior más un sesgo. Siguiendo el ejercicio visto, vamos a crear una red neuronal con la misma configuración:
- Dos capas tipo `Dense`, con 300 y 100 unidades, respectivamente, ambas con función de activación ReLU.
- Una última capa (salida) de 1 unidad, con función de activación `sigmoide`. Tiene una sola unidad porque al usar sigmoide, devolverá un valor entre 0 y 1.


In [23]:
model = keras.models.c()
#model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

Con el método `compile` especificamos la función de pérdida, coste (`loss`); el optimizador (`optimizer`) para buscar los pesos adecuados y la métrica (`metrics`) para medir el rendimiento del modelo.

In [24]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

Ahora podemos entrenar el modelo mediante el método `fit` que recibe el el conjunto de entrenamiento, el número de *epochs* y un conjunto de validación para ir midiendo el rendimiento durante el proceso. Este último es opcional, pero vamos a utilizarlo para seguir el ejemplo visto en la sesión.<br>
Para obtener este conjunto de validación vamos a coger 8000 ejemplos para el conjunto de entrenamiento y otros 8000 para el de validación.<br>
Como resultado vamos a guardar en `history` las estadísticas obtenidas en cada epoch.

In [25]:
X_valid, X_train = X_train[:8000], X_train[8000:]
y_valid, y_train = y_train[:8000], y_train[8000:]
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Epoch 1/30


/usr/local/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:448: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential_7/dense_21/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential_7/dense_21/embedding_lookup_sparse/Reshape:0", shape=(None, 300), dtype=float32), dense_shape=Tensor("gradient_tape/sequential_7/dense_21/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
2022-01-04 15:01:55.169315: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 0 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1


InvalidArgumentError:  Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 0 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits
 (defined at /usr/local/lib/python3.9/site-packages/keras/backend.py:5113)
]] [Op:__inference_train_function_99649]

Errors may have originated from an input operation.
Input Source operations connected to node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits:
In[0] sparse_categorical_crossentropy/Reshape_1 (defined at /usr/local/lib/python3.9/site-packages/keras/backend.py:5109)	
In[1] sparse_categorical_crossentropy/Reshape (defined at /usr/local/lib/python3.9/site-packages/keras/backend.py:3561)

Operation defined at: (most recent call last)
>>>   File "/usr/local/Cellar/python@3.9/3.9.9/Frameworks/Python.framework/Versions/3.9/lib/python3.9/runpy

In [10]:
model.evaluate(X_test, y_test)

782/782 [==============================] - 2s 3ms/step - loss: 0.4359 - accuracy: 0.8615


[0.435902863740921, 0.8614799976348877]

## Parte 2: aplicación de redes neuronales a regresión (predicción del precio de la vivienda)

Se pide aplicar un modelo de redes neuronales al problema de predecir precios de vivienda usando el conjunto de datos  `Boston house prices`. 

Hacerlo usando los dos sistemas vistos, comparando los resultados:
* Scikit learn: usar `MLPRegressor`
* Keras con Tensorflow: usar nuevamente `Sequential` y capas tipo `Dense` con la arquitectura adecuada.

El conjunto de datos se puede cargar usando tanto scikit learn (`sklearn.datasets.load_boston`) como keras (`keras.datasets.boston_housing`). 

Como en la parte 1, se pide mostrar algunas pruebas de los resultados obtenidos usando distintas arquitecturas y/o hiperparámetros. 
